# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [171]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=ywang password=Love520!")
cur = conn.cursor()


In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
pwd

'/home/noma/Documents/OpenClass/Udacity/DEND/Udacity_DEND/SampleData/project_template'

In [6]:
ls

create_tables.py  etl_local.ipynb  __pycache__/  sql_queries.py
etl.ipynb         etl.py           README.md     test.ipynb


In [7]:
song_files = get_files(filepath)

In [4]:
filepath = '../song_data'

In [141]:
song_files

['/home/noma/Documents/OpenClass/Udacity/DEND/Udacity_DEND/SampleData/song_data/TRAAAAW128F429D538.json']

In [181]:
df = pd.read_json(song_files[0],lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [191]:
song_data = df[['song_id','title','year','artist_id','duration']].values[0].tolist()
#song_data = tuple(song_data)
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 0, 'ARD7TVE1187B99BFB1', 218.93179]

In [2]:
run create_tables.py

In [170]:
run sql_queries.py

In [10]:
song_table_insert

' INSERT INTO song (song_id, title, year, artist_id, duration ) VALUES (%s, %s, %s, %s, %s)\n'

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [14]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [18]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [35]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [42]:
log_files = get_files(filepath)

In [41]:
filepath = '../log_data'

In [43]:
log_files

['/home/noma/Documents/OpenClass/Udacity/DEND/Udacity_DEND/SampleData/log_data/2018-11-01-events.json',
 '/home/noma/Documents/OpenClass/Udacity/DEND/Udacity_DEND/SampleData/log_data/2018-11-06-events.json']

In [208]:
df = pd.read_json(log_files[1],lines=True)
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Adler,M,0,Barrera,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540836e+12,248,None,200,1541470364796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
1,Gustavo Cerati,Logged In,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Uno Entre 1000,200,1541470383796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
2,Limp Bizkit,Logged In,Adler,M,2,Barrera,270.49751,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Behind Blue Eyes,200,1541470632796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
3,None,Logged In,Samuel,M,0,Gonzalez,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540493e+12,252,None,200,1541473967796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
4,Mikel Erentxun,Logged In,Samuel,M,1,Gonzalez,178.83383,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,252,Frases Mudas,200,1541474048796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
5,The Gerbils,Logged In,Martin,M,0,Johnson,27.01016,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,(iii),200,1541480171796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55
6,AFI,Logged In,Martin,M,1,Johnson,190.45832,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,Girl's Not Grey,200,1541480198796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55
7,None,Logged In,Jordan,F,0,Hicks,NaN,free,"Salinas, CA",GET,Home,1.540009e+12,240,None,200,1541480984796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
8,Lhasa De Sela,Logged In,Jordan,F,1,Hicks,256.10404,free,"Salinas, CA",PUT,NextSong,1.540009e+12,240,De cara a la pared,200,1541481077796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
9,J.J. Cale,Logged In,Jordan,F,2,Hicks,150.80444,free,"Salinas, CA",PUT,NextSong,1.540009e+12,240,Crazy Mama,200,1541481333796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [52]:
df = df[df.page == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Gustavo Cerati,Logged In,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Uno Entre 1000,200,1541470383796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
2,Limp Bizkit,Logged In,Adler,M,2,Barrera,270.49751,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Behind Blue Eyes,200,1541470632796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
4,Mikel Erentxun,Logged In,Samuel,M,1,Gonzalez,178.83383,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,252,Frases Mudas,200,1541474048796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
5,The Gerbils,Logged In,Martin,M,0,Johnson,27.01016,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,(iii),200,1541480171796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55
6,AFI,Logged In,Martin,M,1,Johnson,190.45832,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,Girl's Not Grey,200,1541480198796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55


In [108]:
t = (pd.Timestamp(i) for i in df.ts.values)
#t = [(i,i.hour,i.day,i.weekofyear,i.month,i.year,i.dayofweek) for i in t]
time_data = [(i,i.hour,i.day,i.weekofyear,i.month,i.year,i.dayofweek) for i in t]

In [109]:

column_labels = ('timestamp','hour','day','week of year','month','year','weekday')

In [110]:
time_df = pd.DataFrame(time_data,columns=column_labels)
time_df.head(10)

,timestamp,hour,day,week of year,month,year,weekday
0,1970-01-01 00:25:41.470383796,0,1,1,1,1970,3
1,1970-01-01 00:25:41.470632796,0,1,1,1,1970,3
2,1970-01-01 00:25:41.474048796,0,1,1,1,1970,3
3,1970-01-01 00:25:41.480171796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.480198796,0,1,1,1,1970,3
5,1970-01-01 00:25:41.481077796,0,1,1,1,1970,3
6,1970-01-01 00:25:41.481333796,0,1,1,1,1970,3
7,1970-01-01 00:25:41.481483796,0,1,1,1,1970,3
8,1970-01-01 00:25:41.481748796,0,1,1,1,1970,3
9,1970-01-01 00:25:41.484351796,0,1,1,1,1970,3


In [115]:
for i, row in time_df.iterrows():
    print (list(row))

[Timestamp('1970-01-01 00:25:41.470383796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.470632796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.474048796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.480171796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.480198796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.481077796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.481333796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.481483796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.481748796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.484351796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.484560796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.484813796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.485058796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.485312796'), 0, 1, 1, 1, 1970, 3]
[Timestamp('1970-01-01 00:25:41.485351796'), 0, 1, 1, 1, 1970, 3]
[Timestamp

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [121]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [122]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Gustavo Cerati,Logged In,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Uno Entre 1000,200,1541470383796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
2,Limp Bizkit,Logged In,Adler,M,2,Barrera,270.49751,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Behind Blue Eyes,200,1541470632796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
4,Mikel Erentxun,Logged In,Samuel,M,1,Gonzalez,178.83383,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,252,Frases Mudas,200,1541474048796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
5,The Gerbils,Logged In,Martin,M,0,Johnson,27.01016,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,(iii),200,1541480171796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55
6,AFI,Logged In,Martin,M,1,Johnson,190.45832,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,Girl's Not Grey,200,1541480198796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55


In [126]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df.head()

,userId,firstName,lastName,gender,level
1,100,Adler,Barrera,M,free
2,100,Adler,Barrera,M,free
4,61,Samuel,Gonzalez,M,free
5,55,Martin,Johnson,M,free
6,55,Martin,Johnson,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [127]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [140]:
df.head(10)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Gustavo Cerati,Logged In,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Uno Entre 1000,200,1541470383796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
2,Limp Bizkit,Logged In,Adler,M,2,Barrera,270.49751,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Behind Blue Eyes,200,1541470632796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
4,Mikel Erentxun,Logged In,Samuel,M,1,Gonzalez,178.83383,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,252,Frases Mudas,200,1541474048796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
5,The Gerbils,Logged In,Martin,M,0,Johnson,27.01016,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,(iii),200,1541480171796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55
6,AFI,Logged In,Martin,M,1,Johnson,190.45832,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,Girl's Not Grey,200,1541480198796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55
8,Lhasa De Sela,Logged In,Jordan,F,1,Hicks,256.10404,free,"Salinas, CA",PUT,NextSong,1.540009e+12,240,De cara a la pared,200,1541481077796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
9,J.J. Cale,Logged In,Jordan,F,2,Hicks,150.80444,free,"Salinas, CA",PUT,NextSong,1.540009e+12,240,Crazy Mama,200,1541481333796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
10,Brian Eno,Logged In,Jordan,F,3,Hicks,265.82159,free,"Salinas, CA",PUT,NextSong,1.540009e+12,240,An Ending (Ascent) (2005 Digital Remaster),200,1541481483796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
11,Hot Hot Heat,Logged In,Jordan,F,4,Hicks,190.01424,free,"Salinas, CA",PUT,NextSong,1.540009e+12,240,Soldier In A Box (Album Version),200,1541481748796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
13,ATB,Logged In,Ryan,M,0,Smith,209.26649,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,278,You're Not Alone,200,1541484351796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [175]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    #insert songplay record
    songplay_data = (songid,pd.Timestamp(row['ts']),row['userId'],row['level'],artistid,row['sessionId'],row['location'],row['userAgent'])
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [206]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [207]:
run create_tables.py

ObjectInUse: database "sparkifydb" is being accessed by other users
DETAIL:  There are 4 other sessions using the database.
